In [1]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
import sys
sys.path.append('../')
from user_packages import hashing


In [2]:
# set variables
target_model_name = 'excited.skirt.earth'
source_file_name = 'OH_repair_columns_analysis.csv'
source_file_path = ''

record_source = 'SQLServerAnalysis'
SystemKeyPhrase = 'OpenHousing' # this should come from the System Hub


In [3]:
# db connect
conn = sqlite3.connect('../full_metadata.db')

In [5]:
# read table to df
dtypeDict = {
    'CHARACTER_MAXIMUM_LENGTH': 'Int64'
  , 'CHARACTER_OCTET_LENGTH': 'Int64'
  , 'NUMERIC_PRECISION': 'Int64'
  , 'NUMERIC_PRECISION_RADIX': 'Int64'
  , 'NUMERIC_SCALE': 'Int64'
  , 'DATETIME_PRECISION': 'Int64'
}
#, nrows=10
df = pd.read_csv(source_file_name, dtype = dtypeDict)
df = df.replace({np.nan: None})


df

,SERVER_NAME,DATABASE_NAME,SCHEMA_NAME,TABLE_NAME,COLUMN_NAME,ORDINAL_POSITION,COLUMN_DEFAULT,IS_NULLABLE,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,...,CHARACTER_SET_CATALOG,CHARACTER_SET_SCHEMA,CHARACTER_SET_NAME,COLLATION_CATALOG,COLLATION_SCHEMA,COLLATION_NAME,DOMAIN_CATALOG,DOMAIN_SCHEMA,DOMAIN_NAME,COLUMN_DESCRIPTION
0,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,ih_provision_category,prrowid,1,None,NO,varchar,36,...,None,None,iso_1,None,None,Latin1_General_CI_AS,None,None,None,None
1,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,ih_provision_category,org_code,2,None,YES,varchar,16,...,None,None,iso_1,None,None,Latin1_General_CI_AS,None,None,None,None
2,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,ih_provision_category,category,3,None,YES,varchar,20,...,None,None,iso_1,None,None,Latin1_General_CI_AS,None,None,None,None
3,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,ih_provision_category,description,4,None,YES,varchar,60,...,None,None,iso_1,None,None,Latin1_General_CI_AS,None,None,None,None
4,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,ih_provision_category,Pro2SrcPDB,5,None,YES,varchar,12,...,None,None,iso_1,None,None,Latin1_General_CI_AS,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,rm_job,sub_sub_category_code,125,None,YES,varchar,24,...,None,None,iso_1,None,None,Latin1_General_CI_AS,None,None,None,None
1087,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,rm_job,usage_code,126,None,YES,varchar,16,...,None,None,iso_1,None,None,Latin1_General_CI_AS,None,None,None,None
1088,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,rm_job,Pro2SrcPDB,127,None,YES,varchar,12,...,None,None,iso_1,None,None,Latin1_General_CI_AS,None,None,None,None
1089,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,rm_job,pro2created,128,None,YES,datetime,None,...,None,None,None,None,None,None,None,None,None,None


In [6]:
# add extra columns

# columns with variable data
df['RecordSource'] = record_source
df['LoadDate'] = datetime.now()
df['SystemKeyPhrase'] = SystemKeyPhrase


# derive keyphrase columns
df['PhysicalAttributeKeyPhrase'] = df.apply(
  lambda row: 
    row['SystemKeyPhrase']
    + '.' + row['SERVER_NAME']
    + '.' + row['DATABASE_NAME']
    + '.' + row['SCHEMA_NAME']
    + '.' + row['TABLE_NAME']
    + '.' + row['COLUMN_NAME']
  , axis=1
)


# hash the keyphrases
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'PhysicalAttributeHashKey'
  , columns = ['PhysicalAttributeKeyPhrase']
)


# hash the payload
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'HashDiff'
  , columns = [
      'SERVER_NAME'
    , 'DATABASE_NAME'
    , 'SCHEMA_NAME'
    , 'TABLE_NAME'
    , 'COLUMN_NAME'

    , 'ORDINAL_POSITION'
    , 'COLUMN_DEFAULT'
    , 'IS_NULLABLE'
    , 'DATA_TYPE'
    , 'CHARACTER_MAXIMUM_LENGTH'
    , 'CHARACTER_OCTET_LENGTH'
    , 'NUMERIC_PRECISION'
    , 'NUMERIC_PRECISION_RADIX'
    , 'NUMERIC_SCALE'
    , 'DATETIME_PRECISION'
    , 'CHARACTER_SET_CATALOG'
    , 'CHARACTER_SET_SCHEMA'
    , 'CHARACTER_SET_NAME'
    , 'COLLATION_CATALOG'
    , 'COLLATION_SCHEMA'
    , 'COLLATION_NAME'
    , 'DOMAIN_CATALOG'
    , 'DOMAIN_SCHEMA'
    , 'DOMAIN_NAME'
    , 'COLUMN_DESCRIPTION'
  ]
)

df

,SERVER_NAME,DATABASE_NAME,SCHEMA_NAME,TABLE_NAME,COLUMN_NAME,ORDINAL_POSITION,COLUMN_DEFAULT,IS_NULLABLE,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,...,DOMAIN_CATALOG,DOMAIN_SCHEMA,DOMAIN_NAME,COLUMN_DESCRIPTION,RecordSource,LoadDate,SystemKeyPhrase,PhysicalAttributeKeyPhrase,PhysicalAttributeHashKey,HashDiff
0,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,ih_provision_category,prrowid,1,None,NO,varchar,36,...,None,None,None,None,SQLServerAnalysis,2024-05-31 08:54:23.821539,OpenHousing,OpenHousing.VHLOPRHP2S01\P2SLIVE.Pro2_ih.dbo.i...,4b992fbf705e9f1e89dfafc88964faa1,a59b3e8de5bf5091ee443f0a8a5d3ba5
1,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,ih_provision_category,org_code,2,None,YES,varchar,16,...,None,None,None,None,SQLServerAnalysis,2024-05-31 08:54:23.821539,OpenHousing,OpenHousing.VHLOPRHP2S01\P2SLIVE.Pro2_ih.dbo.i...,eb8674e5e129cecc1127d794ad0fa76a,09e5fa83dcc654a6ede043c167368d80
2,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,ih_provision_category,category,3,None,YES,varchar,20,...,None,None,None,None,SQLServerAnalysis,2024-05-31 08:54:23.821539,OpenHousing,OpenHousing.VHLOPRHP2S01\P2SLIVE.Pro2_ih.dbo.i...,a147a0ccbe2df0a2ff2be3005b4d6492,157aa32128bdd9047f6baf34d196054e
3,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,ih_provision_category,description,4,None,YES,varchar,60,...,None,None,None,None,SQLServerAnalysis,2024-05-31 08:54:23.821539,OpenHousing,OpenHousing.VHLOPRHP2S01\P2SLIVE.Pro2_ih.dbo.i...,e9de024464239567eb1884734ec1d2fb,8362ade952d3f0b64f2495215b9a989b
4,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,ih_provision_category,Pro2SrcPDB,5,None,YES,varchar,12,...,None,None,None,None,SQLServerAnalysis,2024-05-31 08:54:23.821539,OpenHousing,OpenHousing.VHLOPRHP2S01\P2SLIVE.Pro2_ih.dbo.i...,ad8b7315153220c0c21a0f6de98b6fc0,0628f5df2a703a278e56b5a4c8c64a13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,rm_job,sub_sub_category_code,125,None,YES,varchar,24,...,None,None,None,None,SQLServerAnalysis,2024-05-31 08:54:23.821539,OpenHousing,OpenHousing.VHLOPRHP2S01\P2SLIVE.Pro2_ih.dbo.r...,b13ea0245a4d47c9a84d5856127aa2e7,b878ef1d48d5f2e110c42e0c0cfb4178
1087,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,rm_job,usage_code,126,None,YES,varchar,16,...,None,None,None,None,SQLServerAnalysis,2024-05-31 08:54:23.821539,OpenHousing,OpenHousing.VHLOPRHP2S01\P2SLIVE.Pro2_ih.dbo.r...,d85ca56c4be3f94ac60aef63a6e7358c,404be158183a69f24aaeb5d8167a5be1
1088,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,rm_job,Pro2SrcPDB,127,None,YES,varchar,12,...,None,None,None,None,SQLServerAnalysis,2024-05-31 08:54:23.821539,OpenHousing,OpenHousing.VHLOPRHP2S01\P2SLIVE.Pro2_ih.dbo.r...,1541c7074c9b777447d1a42712cb21d2,8c854e55eb0a9dd08a2ad03f985c5558
1089,VHLOPRHP2S01\P2SLIVE,Pro2_ih,dbo,rm_job,pro2created,128,None,YES,datetime,None,...,None,None,None,None,SQLServerAnalysis,2024-05-31 08:54:23.821539,OpenHousing,OpenHousing.VHLOPRHP2S01\P2SLIVE.Pro2_ih.dbo.r...,3fc416991785c234e6ecc670dcc0b4d4,3544cfc5983836360eef947784857063


In [12]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_SqlServerCsvToPhysicalAttribute")
conn.commit()
df.to_sql('stg_Py_SqlServerCsvToPhysicalAttribute', conn, if_exists='append', index=False)

1091

In [45]:
# Write to the Hub (PhysicalStructure)
# NO, we are not creating new structures

In [13]:
# Write to the Satellite (PhysicalAttribute_SqlServerScrape)
sql_query = """
INSERT INTO rv_s_PhysicalAttribute_SqlServerScrape
(
    PhysicalAttributeHashKey
  , LoadDate
  , RecordSource
  , HashDiff

  , SERVER_NAME
  , "DATABASE_NAME"
  , SCHEMA_NAME
  , TABLE_NAME
  , COLUMN_NAME

  , ORDINAL_POSITION
  , COLUMN_DEFAULT
  , IS_NULLABLE
  , DATA_TYPE
  , CHARACTER_MAXIMUM_LENGTH
  , CHARACTER_OCTET_LENGTH
  , NUMERIC_PRECISION
  , NUMERIC_PRECISION_RADIX
  , NUMERIC_SCALE
  , DATETIME_PRECISION
  , CHARACTER_SET_CATALOG
  , CHARACTER_SET_SCHEMA
  , CHARACTER_SET_NAME
  , COLLATION_CATALOG
  , COLLATION_SCHEMA
  , COLLATION_NAME
  , DOMAIN_CATALOG
  , DOMAIN_SCHEMA
  , DOMAIN_NAME
  , COLUMN_DESCRIPTION

)
SELECT DISTINCT
    stg.PhysicalAttributeHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.HashDiff
  
  , stg.SERVER_NAME
  , stg."DATABASE_NAME"
  , stg.SCHEMA_NAME
  , stg.TABLE_NAME
  , stg.COLUMN_NAME

  , stg.ORDINAL_POSITION
  , stg.COLUMN_DEFAULT
  , stg.IS_NULLABLE
  , stg.DATA_TYPE
  , stg.CHARACTER_MAXIMUM_LENGTH
  , stg.CHARACTER_OCTET_LENGTH
  , stg.NUMERIC_PRECISION
  , stg.NUMERIC_PRECISION_RADIX
  , stg.NUMERIC_SCALE
  , stg.DATETIME_PRECISION
  , stg.CHARACTER_SET_CATALOG
  , stg.CHARACTER_SET_SCHEMA
  , stg.CHARACTER_SET_NAME
  , stg.COLLATION_CATALOG
  , stg.COLLATION_SCHEMA
  , stg.COLLATION_NAME
  , stg.DOMAIN_CATALOG
  , stg.DOMAIN_SCHEMA
  , stg.DOMAIN_NAME
  , stg.COLUMN_DESCRIPTION

FROM
  stg_Py_SqlServerCsvToPhysicalAttribute AS stg
  LEFT OUTER JOIN rv_s_PhysicalAttribute_SqlServerScrape AS sat ON (
    stg.PhysicalAttributeHashKey = sat.PhysicalAttributeHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_PhysicalAttribute_SqlServerScrape AS z
      WHERE z.PhysicalAttributeHashKey = sat.PhysicalAttributeHashKey
    )
  )
WHERE
  (
    sat.HashDiff != stg.HashDiff
    OR sat.HashDiff IS NULL
  )
""";
conn.execute(sql_query)
conn.commit()

In [ ]:
# No inserting to links - this type of sheet shouldn't have created new Attributes

In [ ]:
# Then it all becomes a bit of a hack, because what we want to do now isn't really determined by what's in the sheet.
# Plus, we are working with what is effectively an effectivity satellite on a hub...

# Set the 'PII' tag onto those tables that are reference
sql_query = """
WITH
current_tags AS ( -- gets the active tags that are on the Attribute in the stg table
  SELECT
      sat.PhysicalAttributeHashKey
    , sat.Tag
    , sat.StartDate
    , sat.EndDate
  FROM
    rv_s_PhysicalAttribute_AnalysisTag sat
    INNER JOIN stg_Py_SqlServerCsvToPhysicalAttribute stg ON (
      sat.PhysicalAttributeHashKey = stg.PhysicalAttributeHashKey
      AND sat.LoadDate = (
        SELECT MAX(z.LoadDate)
        FROM rv_s_PhysicalAttribute_AnalysisTag AS z
        WHERE z.PhysicalAttributeHashKey = sat.PhysicalAttributeHashKey
      )
    )
  WHERE
    sat.EndDate IS NULL
)
INSERT INTO rv_s_PhysicalAttribute_AnalysisTag
(
    PhysicalAttributeHashKey
  , LoadDate
  , RecordSource

  , Tag
  , StartDate

)
SELECT DISTINCT
    stg.PhysicalAttributeHashKey
  , stg.LoadDate
  , stg.RecordSource
  
  , 'PII'
  , stg.LoadDate

FROM
  stg_Py_SqlServerCsvToPhysicalAttribute AS stg
  LEFT OUTER JOIN current_tags AS sat ON (
    stg.PhysicalAttributeHashKey = sat.PhysicalAttributeHashKey
  )
WHERE
  (
    sat.Tag != 'PII'
    OR sat.Tag IS NULL
  )
  AND stg.COLUMN_DESCRIPTION LIKE '%PII%'
""";
conn.execute(sql_query)
conn.commit()
